<div style='background-color: #ffc154;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Devoir maison</h2>
    <h1>Redimensionnement d'une image par <i>seam carving</i></h1>
</div>

L'objectif de ce devoir est de travailler sur le redimensionnement d'une image via un algorithme de _découpage de couture_ (en anglais, _seam carving_).

Il utilise le module `PIL`, qui peut être installé en exécutant la cellule suivante.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade Pillow

On peut importer la classe `Image` du module `PIL` en exécutant la cellule suivante.

In [ ]:
from PIL import Image

### Chargement et enregistrement d'une image

**(1)** Après avoir testé la fonction `charger_image` et la procédure `enregistrer_image`, écrire leur spécification.

In [ ]:
import requests

def charger_image(url):
    """
    Ouvre une image avec le module PIL.
    - Entrée : url (chaîne, url d'un fichier image)
    - Sortie : img (instance de la classe Image)
    """
    image = Image.open(requests.get(url, stream=True).raw)
    image = image.convert(mode='RGB')
    return image

def enregistrer_image(image, nom_fichier):
    """
    Enregistre une image dans un fichier.
    - Entrées : image (instance de la classe Image),
                nom_fichier (chaîne, nom d'un fichier image)
    Effet de bord : écriture dans un fichier
    """
    image.save(nom_fichier)

In [ ]:
image = charger_image('https://ntoulzac.github.io/Cours-NSI-Terminale/prog_dyn/images/seam_carv_acropole0.png')
enregistrer_image(image, "acropole0.png")
image  # Affichage de l'image

### Calcul de l'_énergie_ des pixels

On souhaite calculer l'_écart_ entre la couleur de deux pixels.

Si les composantes RVB des deux couleurs sont respectivement $(R, V, B)$ et $(R', V', B')$, on appelle _écart_ entre les deux couleurs le nombre :

$$0,2126 \times |R - R'| + 0,7152 \times |V - V'| + 0,0722 \times |B - B'|$$

**(2)** Définir une fonction `ecart_couleurs` qui prend en paramètres d'entrée deux couleurs représentées par des p-puplets de la forme `(r, v, b)` et qui renvoie l'écart entre les deux couleurs.

In [ ]:
def ecart_couleurs(coul1, coul2):
    r1, v1, b1 = coul1
    r2, v2, b2 = coul2
    return 0.2126 * abs(r1 - r2) + 0.7152 * abs(v1 - v2) + 0.0722 * abs(b1 - b2)

On souhaite maintenant calculer l'_énergie_ des pixels d'une image.

On appelle _énergie_ d'un pixel l'_écart_ moyen entre la couleur de ce pixel et la couleur de ses voisins.

**(3)** Définir une fonction `energie_pixel` conformément à la spécification suivante :

In [ ]:
def energie_pixel(image, largeur, hauteur, x, y):
    """
    Calcule l'énergie d'un pixel d'une image.
    - Entrées : image (instance de la classe Image)
                largeur, hauteur (entiers, dimensions de l'image)
                x, y (entiers, coordonnées du pixel dont on calcule l'énergie)
    - Sortie : (flottant)
    """
    coul1 = image.getpixel((x, y))  # couleur du pixel étudié
    somme_ecarts = 0
    nb_voisins = 0
    for a in range(-1, 2):
        for b in range(-1, 2):
            if (a, b) != (0, 0) and (x + a) in range(largeur) and (y + b) in range(hauteur):
                coul2 = image.getpixel((x + a, y + b))  # couleur d'un pixel voisin
                somme_ecarts = somme_ecarts + ecart_couleurs(coul1, coul2)
                nb_voisins = nb_voisins + 1
    return somme_ecarts / nb_voisins

**(4)** Définir une fonction `tableau_energies` conformément à la spécification suivante :

In [ ]:
def tableau_energies(image):
    """
    Détermine, sous forme de tableau à deux dimensions, l'énergie de chaque pixel d'une image.
    - Entrée : image (instance de la classe Image)
    - Sortie : tab (tableau à deux dimensions, dont le nombre de lignes est la hauteur de l'image
                    et le nombre de colonnes la largeur de l'image)
    """
    largeur = image.width
    hauteur = image.height
    tab = []
    for y in range(hauteur):
        lig = []
        for x in range(largeur):
            lig.append(energie_pixel(image, largeur, hauteur, x, y))
        tab.append(lig)
    return tab

On donne la fonction suivante, qui permet d'obtenir une représentation de l'énergie des pixels d'une image. Les pixels qui apparaissent en blanc ont une faible énergie, ceux qui apparaissent en noir ont une forte énergie.

In [ ]:
def image_energies(image):
    tab_en = tableau_energies(image)
    en_max = 0
    for x in range(image.width):
        for y in range(image.height):
            if en_max < tab_en[y][x]:
                en_max = tab_en[y][x]
    image_en = image.copy()
    for x in range(image.width):
        for y in range(image.height):
            rvb = int(255 * (1 - tab_en[y][x] / en_max))
            image_en.putpixel((x, y), (rvb, rvb, rvb))
    return image_en

In [ ]:
image_energies(image)

### Détermination de la couture d'énergie minimale

Déf de _couture_

In [ ]:
def recherche_min_dessous(tab, x, y):
    L = [(tab[y+1][a][0], a) for a in range(x-1, x+2) if a in range(len(tab[y]))]
    val_min, x_min = L[0][0], L[0][1]
    for k in range(1, len(L)):
        if L[k][0] < val_min:
            val_min, x_min = L[k][0], L[k][1]
    return val_min, x_min

Couture minimale avec programmation dynamique

In [ ]:
def couture_min(image):
    largeur = image.width
    hauteur = image.height
    tab_en = tableau_energies(image)
    for x in range(largeur):
        tab_en[hauteur-1][x] = (tab_en[hauteur-1][x], None)
    for y in range(hauteur-2, -1, -1):
        for x in range(largeur):
            val_min, x_min = recherche_min_dessous(tab_en, x, y)
            tab_en[y][x] = (val_min + tab_en[y][x], x_min)
    val_min, x_min = tab_en[0][0][0], 0
    for x in range(1, largeur):
        if tab_en[0][x][0] < val_min:
            val_min, x_min = tab_en[0][x][0], x
    x = x_min
    couture = [(x_min, 0)]
    for y in range(hauteur-1):
        x = tab_en[y][x][1]
        couture.append((x, y+1))
    return couture

### Redimensionnement de l'image

Suppression d'une couture

In [ ]:
def retirer_couture(image, couture):
    largeur = image.width
    hauteur = image.height
    for (x, y) in couture:
        for xx in range(x, largeur - 1):
            image.putpixel((xx, y), image.getpixel((xx+1, y)))
    return image.crop((0, 0, largeur-1, hauteur))

### Test

In [ ]:
image = charger_image('https://ntoulzac.github.io/Cours-NSI-Terminale/prog_dyn/images/seam_carv_acropole0.png')
image  # Affichage de l'image

In [ ]:
for _ in range(20):
    image = retirer_couture(image, couture_min(image))

In [ ]:
image